## OBJECTIVES

The main objective of this notebook is to demonstrate the application of scraping techniques, Natural Language Processing (NLP) pipeline, and various NLP tasks such as text cleaning, tokenization, stop words removal, discretization, normalization, stemming, lemmatization, parts-of-speech tagging, and Named Entity Recognition (NER).

- **Author: Omar Nouih**

## TASKS


#### Importing necessary :

In [151]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer
import qalsadi.lemmatizer
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer

### Scraping from Web Sources
<h4>Scraping from : <p style="color:red;"> www.candidature-recensement.ma</p></h4>

>This website provides information about the census process in Morocco. It includes details about the conditions and steps involved in applying for the census. We'll be scraping data related to census conditions, steps, compensation, taches and questions  from this website.

In [152]:
url = "https://www.candidature-recensement.ma/condition.html"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
recensement_conditions = {}
divs_col_md_12 = soup.find_all('div', class_='col-md-12')
for div_col_md_12 in divs_col_md_12:
    p_tags = div_col_md_12.find_all('p')
    for p_tag in p_tags:
        text_content = p_tag.get_text(strip=True)
        if text_content not in recensement_conditions:
            recensement_conditions[text_content] = None
            
recensement_conditions

{'طبقا لمقتضيات المرسوم رقم 2.23.1065 الصادر في 7 جمادى الآخرة 1445 ( 21 ديسمبر 2023)، يؤهل للمشاركة في تهييئ وإنجاز إحصاء السكان والسكنى الأشخاص الذين يتم انتقاؤهم، في حدود الأعداد المطلوبة، من بين:': None,
 'الطلبةأو متدربي المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني الذين تابعوا دراستهم بها برسم السنتين الأولى والثانية، على الأقل، بعد حصولهم على شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.': None,
 'الحاصلين على الدبلومات والشهادات المسلمة من طرف المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني، بعد سنتين من التكوين على الأقل بعد شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.': None,
 'موظفي وأعوان الإدارات العمومية والجماعات الترابية ومستخدمي المؤسسات العمومية وكذا الأشخاص المتقاعدين منهم. (بالنسبة للموظفين والمستخدمين، يشترط الحصول على ترخيص للمشاركة في عملية الإحصاء مسلم من ال

In [153]:
url = "https://www.candidature-recensement.ma/etapes.html"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
recensement_etapes = {}
twms = soup.find('div', class_='twm-step-section-4')
list_items = twms.find_all('li')
for item in list_items:
    title = item.find('h4', class_='twm-title').get_text(strip=True)
    paragraph = item.find('p').get_text(strip=True).replace('\r', '').replace('\n', '').replace('\t', '').strip()
    recensement_etapes[title] = paragraph
recensement_etapes

{'التسجيل عبر المنصة الإلكترونية': 'على المترشحين الراغبين في المشاركة في هذه العملية والمستوفين للشروط الضرورية تعبئة استمارة الترشيح الإلكترونية المتاحة لهم فقط على هذه المنصة.',
 'الانتقاء الأولي': 'سيتم إجراء الانتقاء الأولي، في حدود الأعداد المطلوبة من الموارد البشرية حسب الجماعات الترابية على الصعيد الوطني، بناءاً على ترتيب المترشحين بطريقة أوتوماتيكية بواسطة نظام تقييم محدد مسبقًا على المنصة الإلكترونية وذلك اعتمادا على المعلومات المصرح بها في طلب الترشيح.وسيتم إخبار المترشحين المقبولين في الانتقاء الأولي عبر بريدهم الإلكتروني المصرح به بعد التسجيل.',
 'التكوين عن بعد': 'تتم دعوة المترشحين الذين تم قبولهم في الانتقاء الأولي عبر البريد الإلكتروني المصرح به إلى متابعة، بصفة حتمية، دورة تكوينية عن بعد، عبر منصة محدثة لذلك. ويخضعون من خلالها لتقييم يؤهلهم لإجراء المقابلة الانتقائية. تحتوي هذه الدورة على ثلاث وحدات',
 'المقابلة الانتقائية': 'تتم، عبر البريد الإلكتروني المصرح به، دعوة المترشحين الذين تم انتقاؤهم بناء على التقييم الذي خضعوا له في الدورة التكوينية عن بعد، في حدود الأعدا

In [154]:
url = "https://www.candidature-recensement.ma/taches.html"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
recensement_taches = {}

team_items = soup.find_all('div', class_='team__item')
for team_item in team_items:
    title_element = team_item.find('h4', class_='twm-title')
    text_element = team_item.find('div', class_='team__item__footer__content')
    
    if title_element and text_element:
        title = title_element.text.strip()
        text = text_element.text.strip().replace('\n', '').replace(title, '')
        recensement_taches[title] = text
    
recensement_taches

{'تجميع المعطيات لدى الاسر': 'التعرف على حدود منطقة الإحصاء، باستعمال الخريطة الرقمية، وتحيين المعلومات الجغرافية المكونة لهاالقيام بجولة شاملة داخل منطقة الإحصاء، بدءاً من نقطة انطلاق يتم تحديدها، وذلك من أجل التعرف على مكونات منطقة الإحصاء (جزيرات، بنايات، شوارع، أزقة، دواوير، طرق، إلخ) وتحيينها عند الاقتضاء.زيارة الأسر وفق الجولة التي سيتم تحديدها خلال مرحلة التعرف على الميدان، واستقاء المعطيات المطلوبة في الاستمارات المحملة على اللوحات الالكترونية، سواء المتعلقة بالأسر أو بمؤسسات السكان المحسوبين على حدة والأشخاص بدون مأوى والأشخاص الرحل.العمل على احترام خصوصية الأسر والساكنة والحفاظ باستمرار على اللياقة اللازمة وحسن السلوك عند الاستجواب.التقييد بالمقتضيات القانونية التي تضمن سرية المعلومات الفردية للأسر ولأعضائها وعدم إفشائها لأي طرف كيفما كان، تحت طائلة التعرض للعقوبات الزجرية الواردة في هذا الشأن.العمل على تحقيق شمولية الإحصاء للأسر والسكان بالتأكد من إحصاء كافة الاسر القاطنة بمنطقة الإحصاء وكذلك التأكد من جرد جميع الأفراد داخل كل أسرة، وذلك بالتقيد بالتعليمات الواردة في هذا الش

In [155]:
url = "https://www.candidature-recensement.ma/compensation.html"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
item = soup.find('div', class_='team__item__footer__content')
recensement_compensation = item.find('p').get_text(strip=True).replace('\r', '').replace('\n', '').replace('\t', '').strip()
recensement_compensation

'المشاركون الذين اجتازوا مراحل الانتقاء الأولي، التكوين عن بعد، المقابلة الحضورية وتمت دعوتهم لإنجاز الإحصاء، سيستفيدون من تعويض يومي، حسب المهام المسندة لهم، خلال مرحلة إجراء الإحصاء من 1 إلى 30 شتنبر 2024 (30 يوم عمل) يتراوح ما بين 250 درهم و500 درهم في اليوم، بالإضافة إلى تعويض عن مرحلة التكوين الحضوري، يتراوح ما بين 200 درهم و500 درهم في اليوم، وذلك طبقا لمقتضيات المرسوم رقم 2.23.580 الصادر بتاريخ 21 ديسمبر 2023 بمنح تعويض للمشاركين في تهييئ وإنجاز إحصاء السكان والسكنى في المملكة.'

In [156]:
url = "https://www.candidature-recensement.ma/question.html"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')
recensement_questions = {}
faq_container = soup.find('div', class_='faq-container')
questions = faq_container.find_all('div', class_='question')
for idx, question in enumerate(questions, start=1):
    title_element = question.find('h5')
    text_element = question.find('div', class_='answer')
    
    if title_element and text_element:
        title = title_element.get_text(strip=True).replace('\r', '').replace('\n', '').replace('\t', '').replace('▼','').strip()
        text = text_element.get_text(strip=True).replace('\r', '').replace('\n', '').replace('\t', '').strip()
        recensement_questions[title] = text
recensement_questions

{'من هم الأشخاص المعنيون بالمشاركة في فريق العمل المكلف بإجراء الإحصاء العام للسكان والسكنى 2024 ؟': 'الأشخاص المعنيون هم الأشخاص الذين يتم انتقاؤهم، في حدود الأعداد المطلوبة، من بين :الحاصلين على الدبلومات والشهادات المسلمة من طرف المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني، بعد سنتين من التكوين على الأقل بعد شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.الطلبة أو متدربي المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني الذين تابعوا دراستهم بها برسم السنتين الأولى والثانية، على الأقل، بعد حصولهم على شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.موظفي وأعوان الإدارات العمومية والجماعات الترابية ومستخدمي المؤسسات العمومية وكذا الأشخاص المتقاعدين منهم.',
 'ما هي الوثائق المطلوب الإدلاء بها للمشاركة ؟': 'الوثائق التي يجب توفرها والإدلاء بها للمشاركة في عملية الإحصاء هي كالتالي :بطاقة التع

In [157]:
Recensement = {
    "شروط الترشيح": list(recensement_conditions.keys()),
    "مراحل إنتقاء المترشحين": recensement_etapes,
    "مهام المشاركين " : recensement_taches,
    "تعويضات المشاركين" : recensement_compensation,
    "أسئلة متداولة" : recensement_questions
}

Recensement

{'شروط الترشيح': ['طبقا لمقتضيات المرسوم رقم 2.23.1065 الصادر في 7 جمادى الآخرة 1445 ( 21 ديسمبر 2023)، يؤهل للمشاركة في تهييئ وإنجاز إحصاء السكان والسكنى الأشخاص الذين يتم انتقاؤهم، في حدود الأعداد المطلوبة، من بين:',
  'الطلبةأو متدربي المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني الذين تابعوا دراستهم بها برسم السنتين الأولى والثانية، على الأقل، بعد حصولهم على شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.',
  'الحاصلين على الدبلومات والشهادات المسلمة من طرف المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني، بعد سنتين من التكوين على الأقل بعد شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.',
  'موظفي وأعوان الإدارات العمومية والجماعات الترابية ومستخدمي المؤسسات العمومية وكذا الأشخاص المتقاعدين منهم. (بالنسبة للموظفين والمستخدمين، يشترط الحصول على ترخيص للمشاركة في عملية الإحصاء مسلم من 

### Storing Raw Data - <span style="color:green;">MongoDB</span>

#### Establishing MongoDB Connection and Retrieving Data
> Establishing a connection to MongoDB running locally and retrieving a document from the "NLP" database's "atelier" collection.

In [158]:
client = MongoClient("mongodb://localhost:27017")
db = client["NLP"]
collection = db["atelier"]
# collection.insert_one(Recensement)

In [159]:
document = collection.find_one()
print(document)

{'_id': ObjectId('660ec7f973756d2e9a23cd22'), 'شروط الترشيح': ['طبقا لمقتضيات المرسوم رقم 2.23.1065 الصادر في 7 جمادى الآخرة 1445 ( 21 ديسمبر 2023)، يؤهل للمشاركة في تهييئ وإنجاز إحصاء السكان والسكنى الأشخاص الذين يتم انتقاؤهم، في حدود الأعداد المطلوبة، من بين:', 'الطلبةأو متدربي المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني الذين تابعوا دراستهم بها برسم السنتين الأولى والثانية، على الأقل، بعد حصولهم على شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.', 'الحاصلين على الدبلومات والشهادات المسلمة من طرف المؤسسات الجامعية ومؤسسات التعليم العالي غير التابعة للجامعات ومؤسسات تكوين الأطر العليا ومؤسسات التكوين المهني، بعد سنتين من التكوين على الأقل بعد شهادة الباكالوريا، شريطة ألا تقل سنهم عن 20 سنة في فاتح يناير من سنة إجراء الإحصاء.', 'موظفي وأعوان الإدارات العمومية والجماعات الترابية ومستخدمي المؤسسات العمومية وكذا الأشخاص المتقاعدين منهم. (بالنسبة للموظفين والمستخدمين، يشترط الحصول على ت

### NLP Pipeline - <span style="color:brown;">Data Preprocessing</span>
> For normalization and text cleaning, we have already completed these processes earlier

**Split the text into individual tokens or words.**

In [160]:
tokenizer = RegexpTokenizer("[\w']+")

In [161]:
recensement_conditions_words = []
for text in recensement_conditions.keys():
    tokens = tokenizer.tokenize(text)
    recensement_conditions_words.extend(tokens)

print(recensement_conditions_words)

['طبقا', 'لمقتضيات', 'المرسوم', 'رقم', '2', '23', '1065', 'الصادر', 'في', '7', 'جمادى', 'الآخرة', '1445', '21', 'ديسمبر', '2023', 'يؤهل', 'للمشاركة', 'في', 'تهييئ', 'وإنجاز', 'إحصاء', 'السكان', 'والسكنى', 'الأشخاص', 'الذين', 'يتم', 'انتقاؤهم', 'في', 'حدود', 'الأعداد', 'المطلوبة', 'من', 'بين', 'الطلبةأو', 'متدربي', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'غير', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'الذين', 'تابعوا', 'دراستهم', 'بها', 'برسم', 'السنتين', 'الأولى', 'والثانية', 'على', 'الأقل', 'بعد', 'حصولهم', 'على', 'شهادة', 'الباكالوريا', 'شريطة', 'ألا', 'تقل', 'سنهم', 'عن', '20', 'سنة', 'في', 'فاتح', 'يناير', 'من', 'سنة', 'إجراء', 'الإحصاء', 'الحاصلين', 'على', 'الدبلومات', 'والشهادات', 'المسلمة', 'من', 'طرف', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'غير', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'بعد', 'سنتين', 'من', 'التكوين', 'على', 'الأقل', 'بعد

In [162]:
recensement_etapes_words = []
for key in recensement_etapes.keys():
    tokens = tokenizer.tokenize(key)
    recensement_etapes_words.extend(tokens)

for value in recensement_etapes.values():
    tokens = tokenizer.tokenize(value)
    recensement_etapes_words.extend(tokens)

print(recensement_etapes_words)

['التسجيل', 'عبر', 'المنصة', 'الإلكترونية', 'الانتقاء', 'الأولي', 'التكوين', 'عن', 'بعد', 'المقابلة', 'الانتقائية', 'التكوين', 'الحضوري', 'على', 'المترشحين', 'الراغبين', 'في', 'المشاركة', 'في', 'هذه', 'العملية', 'والمستوفين', 'للشروط', 'الضرورية', 'تعبئة', 'استمارة', 'الترشيح', 'الإلكترونية', 'المتاحة', 'لهم', 'فقط', 'على', 'هذه', 'المنصة', 'سيتم', 'إجراء', 'الانتقاء', 'الأولي', 'في', 'حدود', 'الأعداد', 'المطلوبة', 'من', 'الموارد', 'البشرية', 'حسب', 'الجماعات', 'الترابية', 'على', 'الصعيد', 'الوطني', 'بناءا', 'على', 'ترتيب', 'المترشحين', 'بطريقة', 'أوتوماتيكية', 'بواسطة', 'نظام', 'تقييم', 'محدد', 'مسبق', 'ا', 'على', 'المنصة', 'الإلكترونية', 'وذلك', 'اعتمادا', 'على', 'المعلومات', 'المصرح', 'بها', 'في', 'طلب', 'الترشيح', 'وسيتم', 'إخبار', 'المترشحين', 'المقبولين', 'في', 'الانتقاء', 'الأولي', 'عبر', 'بريدهم', 'الإلكتروني', 'المصرح', 'به', 'بعد', 'التسجيل', 'تتم', 'دعوة', 'المترشحين', 'الذين', 'تم', 'قبولهم', 'في', 'الانتقاء', 'الأولي', 'عبر', 'البريد', 'الإلكتروني', 'المصرح', 'به', 'إلى', 

In [163]:
recensement_taches_words = []
for key in recensement_taches.keys():
    tokens = tokenizer.tokenize(key)
    recensement_taches_words.extend(tokens)

for value in recensement_taches.values():
    tokens = tokenizer.tokenize(value)
    recensement_taches_words.extend(tokens)

print(recensement_taches_words)

['تجميع', 'المعطيات', 'لدى', 'الاسر', 'مراقبة', 'جودة', 'المعطيات', 'المجمعة', 'تنظيم', 'العمل', 'في', 'الميدان', 'التعرف', 'على', 'حدود', 'منطقة', 'الإحصاء', 'باستعمال', 'الخريطة', 'الرقمية', 'وتحيين', 'المعلومات', 'الجغرافية', 'المكونة', 'لهاالقيام', 'بجولة', 'شاملة', 'داخل', 'منطقة', 'الإحصاء', 'بدءا', 'من', 'نقطة', 'انطلاق', 'يتم', 'تحديدها', 'وذلك', 'من', 'أجل', 'التعرف', 'على', 'مكونات', 'منطقة', 'الإحصاء', 'جزيرات', 'بنايات', 'شوارع', 'أزقة', 'دواوير', 'طرق', 'إلخ', 'وتحيينها', 'عند', 'الاقتضاء', 'زيارة', 'الأسر', 'وفق', 'الجولة', 'التي', 'سيتم', 'تحديدها', 'خلال', 'مرحلة', 'التعرف', 'على', 'الميدان', 'واستقاء', 'المعطيات', 'المطلوبة', 'في', 'الاستمارات', 'المحملة', 'على', 'اللوحات', 'الالكترونية', 'سواء', 'المتعلقة', 'بالأسر', 'أو', 'بمؤسسات', 'السكان', 'المحسوبين', 'على', 'حدة', 'والأشخاص', 'بدون', 'مأوى', 'والأشخاص', 'الرحل', 'العمل', 'على', 'احترام', 'خصوصية', 'الأسر', 'والساكنة', 'والحفاظ', 'باستمرار', 'على', 'اللياقة', 'اللازمة', 'وحسن', 'السلوك', 'عند', 'الاستجواب', 'التق

In [164]:
recensement_compensation_words = []
tokens = tokenizer.tokenize(recensement_compensation)
recensement_compensation_words.extend(tokens)

print(recensement_compensation_words)

['المشاركون', 'الذين', 'اجتازوا', 'مراحل', 'الانتقاء', 'الأولي', 'التكوين', 'عن', 'بعد', 'المقابلة', 'الحضورية', 'وتمت', 'دعوتهم', 'لإنجاز', 'الإحصاء', 'سيستفيدون', 'من', 'تعويض', 'يومي', 'حسب', 'المهام', 'المسندة', 'لهم', 'خلال', 'مرحلة', 'إجراء', 'الإحصاء', 'من', '1', 'إلى', '30', 'شتنبر', '2024', '30', 'يوم', 'عمل', 'يتراوح', 'ما', 'بين', '250', 'درهم', 'و500', 'درهم', 'في', 'اليوم', 'بالإضافة', 'إلى', 'تعويض', 'عن', 'مرحلة', 'التكوين', 'الحضوري', 'يتراوح', 'ما', 'بين', '200', 'درهم', 'و500', 'درهم', 'في', 'اليوم', 'وذلك', 'طبقا', 'لمقتضيات', 'المرسوم', 'رقم', '2', '23', '580', 'الصادر', 'بتاريخ', '21', 'ديسمبر', '2023', 'بمنح', 'تعويض', 'للمشاركين', 'في', 'تهييئ', 'وإنجاز', 'إحصاء', 'السكان', 'والسكنى', 'في', 'المملكة']


In [165]:
recensement_questions_words = []
for key in recensement_questions.keys():
    tokens = tokenizer.tokenize(key)
    recensement_questions_words.extend(tokens)

for value in recensement_questions.values():
    tokens = tokenizer.tokenize(value)
    recensement_questions_words.extend(tokens)

print(recensement_questions_words)

['من', 'هم', 'الأشخاص', 'المعنيون', 'بالمشاركة', 'في', 'فريق', 'العمل', 'المكلف', 'بإجراء', 'الإحصاء', 'العام', 'للسكان', 'والسكنى', '2024', 'ما', 'هي', 'الوثائق', 'المطلوب', 'الإدلاء', 'بها', 'للمشاركة', 'هل', 'يمكنني', 'الإدلاء', 'بشهادة', 'النجاح', 'عوض', 'الدبلوم', 'كيف', 'يتم', 'انتقائي', 'للمشاركة', 'في', 'عملية', 'الإحصاء', 'بعد', 'وضع', 'طلبي', 'ماذا', 'يتوجب', 'علي', 'فعله', 'متى', 'وكيف', 'سيتم', 'إخباري', 'بوضعية', 'طلب', 'ترشحي', 'الأشخاص', 'المعنيون', 'هم', 'الأشخاص', 'الذين', 'يتم', 'انتقاؤهم', 'في', 'حدود', 'الأعداد', 'المطلوبة', 'من', 'بين', 'الحاصلين', 'على', 'الدبلومات', 'والشهادات', 'المسلمة', 'من', 'طرف', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'غير', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'بعد', 'سنتين', 'من', 'التكوين', 'على', 'الأقل', 'بعد', 'شهادة', 'الباكالوريا', 'شريطة', 'ألا', 'تقل', 'سنهم', 'عن', '20', 'سنة', 'في', 'فاتح', 'يناير', 'من', 'سنة', 'إجراء', 'الإحصاء', 'الطلبة', 'أو', 'متدربي'

In [166]:
words = recensement_questions_words + recensement_compensation_words + recensement_taches_words + recensement_etapes_words + recensement_conditions_words
print(words)
print(f"Number of words : {len(words)}")

['من', 'هم', 'الأشخاص', 'المعنيون', 'بالمشاركة', 'في', 'فريق', 'العمل', 'المكلف', 'بإجراء', 'الإحصاء', 'العام', 'للسكان', 'والسكنى', '2024', 'ما', 'هي', 'الوثائق', 'المطلوب', 'الإدلاء', 'بها', 'للمشاركة', 'هل', 'يمكنني', 'الإدلاء', 'بشهادة', 'النجاح', 'عوض', 'الدبلوم', 'كيف', 'يتم', 'انتقائي', 'للمشاركة', 'في', 'عملية', 'الإحصاء', 'بعد', 'وضع', 'طلبي', 'ماذا', 'يتوجب', 'علي', 'فعله', 'متى', 'وكيف', 'سيتم', 'إخباري', 'بوضعية', 'طلب', 'ترشحي', 'الأشخاص', 'المعنيون', 'هم', 'الأشخاص', 'الذين', 'يتم', 'انتقاؤهم', 'في', 'حدود', 'الأعداد', 'المطلوبة', 'من', 'بين', 'الحاصلين', 'على', 'الدبلومات', 'والشهادات', 'المسلمة', 'من', 'طرف', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'غير', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'بعد', 'سنتين', 'من', 'التكوين', 'على', 'الأقل', 'بعد', 'شهادة', 'الباكالوريا', 'شريطة', 'ألا', 'تقل', 'سنهم', 'عن', '20', 'سنة', 'في', 'فاتح', 'يناير', 'من', 'سنة', 'إجراء', 'الإحصاء', 'الطلبة', 'أو', 'متدربي'

**Eliminate common stop words from the text data.**

In [167]:
print(stopwords.words('arabic'))
stop_words = set(stopwords.words('arabic'))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [168]:
words = [word for word in words if word not in stop_words]
print(words)
print(f"Number of words : {len(words)}")

['الأشخاص', 'المعنيون', 'بالمشاركة', 'فريق', 'العمل', 'المكلف', 'بإجراء', 'الإحصاء', 'العام', 'للسكان', 'والسكنى', '2024', 'الوثائق', 'المطلوب', 'الإدلاء', 'للمشاركة', 'يمكنني', 'الإدلاء', 'بشهادة', 'النجاح', 'الدبلوم', 'يتم', 'انتقائي', 'للمشاركة', 'عملية', 'الإحصاء', 'وضع', 'طلبي', 'يتوجب', 'علي', 'فعله', 'وكيف', 'سيتم', 'إخباري', 'بوضعية', 'طلب', 'ترشحي', 'الأشخاص', 'المعنيون', 'الأشخاص', 'يتم', 'انتقاؤهم', 'حدود', 'الأعداد', 'المطلوبة', 'الحاصلين', 'الدبلومات', 'والشهادات', 'المسلمة', 'طرف', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'سنتين', 'التكوين', 'الأقل', 'شهادة', 'الباكالوريا', 'شريطة', 'تقل', 'سنهم', '20', 'سنة', 'فاتح', 'سنة', 'إجراء', 'الإحصاء', 'الطلبة', 'متدربي', 'المؤسسات', 'الجامعية', 'ومؤسسات', 'التعليم', 'العالي', 'التابعة', 'للجامعات', 'ومؤسسات', 'تكوين', 'الأطر', 'العليا', 'ومؤسسات', 'التكوين', 'المهني', 'تابعوا', 'دراستهم', 'برسم', 'السنتين', 'الأولى', 'وال

### Stemming and Lemmatization 
Apply stemming and lemmatization techniques to the preprocessed text data

**Stemming :**

In [169]:
stemmer = ISRIStemmer()
stemmed_dict = {word: stemmer.stem(word) for word in words}
print("Stemmed Dictionary:")
for word, stemmed_word in stemmed_dict.items():
    print(f"{word}: {stemmed_word}")

Stemmed Dictionary:
الأشخاص: شخص
المعنيون: عني
بالمشاركة: شرك
فريق: فرق
العمل: عمل
المكلف: كلف
بإجراء: إجراء
الإحصاء: حصء
العام: عام
للسكان: سكن
والسكنى: كنى
2024: 2024
الوثائق: وثق
المطلوب: طلب
الإدلاء: دلء
للمشاركة: شرك
يمكنني: يمك
بشهادة: شهد
النجاح: نجح
الدبلوم: دبلوم
يتم: يتم
انتقائي: نقئ
عملية: عمل
وضع: وضع
طلبي: طلب
يتوجب: وجب
علي: علي
فعله: فعل
وكيف: وكف
سيتم: يتم
إخباري: خبر
بوضعية: وضع
طلب: طلب
ترشحي: رشح
انتقاؤهم: نقؤ
حدود: حدد
الأعداد: عدد
المطلوبة: طلب
الحاصلين: حصل
الدبلومات: دبلوم
والشهادات: شهد
المسلمة: سلم
طرف: طرف
المؤسسات: ؤسس
الجامعية: جمع
ومؤسسات: ؤسس
التعليم: علم
العالي: علي
التابعة: ابع
للجامعات: جمع
تكوين: تكو
الأطر: اطر
العليا: علا
التكوين: تكو
المهني: هني
سنتين: سنت
الأقل: اقل
شهادة: شهد
الباكالوريا: كلر
شريطة: شرط
تقل: تقل
سنهم: نهم
20: 20
سنة: سنة
فاتح: فتح
إجراء: جرء
الطلبة: طلب
متدربي: تدرب
تابعوا: تبع
دراستهم: درس
برسم: رسم
السنتين: سنت
الأولى: ولى
والثانية: ثني
حصولهم: حصل
موظفي: وظف
وأعوان: أعو
الإدارات: ادر
العمومية: عمم
والجماعات: جمع
الترابية: ربة
وم

**Lemmatization :**

In [170]:
lemmatizer = qalsadi.lemmatizer.Lemmatizer()
lemmatized_dict = {word: lemmatizer.lemmatize(word) for word in words}
print("Lemmatized  Dictionary:")
for word, lemmatized_word in lemmatized_dict.items():
    print(f"{word}: {lemmatized_word}")

Lemmatized  Dictionary:
الأشخاص: أشخاص
المعنيون: معني
بالمشاركة: مشارك
فريق: ريق
العمل: عمل
المكلف: مكلف
بإجراء: إجراء
الإحصاء: إحصاء
العام: عام
للسكان: سكان
والسكنى: سكنى
2024: 2024
الوثائق: وثائق
المطلوب: مطلوب
الإدلاء: إدلاء
للمشاركة: مشارك
يمكنني: أمكن
بشهادة: شهاد
النجاح: نجاح
الدبلوم: دبلوم
يتم: يتم
انتقائي: انتقاء
عملية: عملة
وضع: وضع
طلبي: طلب
يتوجب: توجب
علي: علي
فعله: فعل
وكيف: كيف
سيتم: أتم
إخباري: إخبار
بوضعية: وضع
طلب: طلب
ترشحي: ترشح
انتقاؤهم: انتقاء
حدود: حدود
الأعداد: أعداد
المطلوبة: مطلوب
الحاصلين: حاصل
الدبلومات: دبلوم
والشهادات: شهادة
المسلمة: مسلم
طرف: طرف
المؤسسات: مؤسس
الجامعية: جامع
ومؤسسات: مؤسس
التعليم: تعليم
العالي: عال
التابعة: تابع
للجامعات: جامع
تكوين: تكوين
الأطر: أطر
العليا: عليا
التكوين: تكوين
المهني: مهن
سنتين: سنة
الأقل: أقل
شهادة: شهاد
الباكالوريا: الباكالوريا
شريطة: شريط
تقل: تقلي
سنهم: سن
20: 20
سنة: سنة
فاتح: فاتح
إجراء: إجراء
الطلبة: طلب
متدربي: متدرب
تابعوا: تابع
دراستهم: دراس
برسم: رسم
السنتين: سنة
الأولى: أولى
والثانية: ثاني
حصولهم: حصول
موظفي:

<h3><span style="color:purple;">Comparison</span> : Stemming vs. Lemmatization</h3>

> When comparing stemming and lemmatization for Arabic words, using lemmatization with the qalsadi library appears to yield more accurate results compared to stemming using the traditional method. Stemming involves stripping words to their root form, often resulting in incomplete or inaccurate representations, especially in languages with rich morphology like Arabic. On the other hand, lemmatization considers the context of words and produces more linguistically accurate outputs. With the qalsadi library, lemmatization can identify the lemma or dictionary form of Arabic words more effectively, preserving their semantic meaning and linguistic nuances. This makes lemmatization a preferred choice for tasks requiring precise language processing, such as natural language understanding and machine translation, particularly in Arabic text analysis.

### Part-of-Speech Tagging and Named Entity Recognition <span style="color:green;">using Farasa</span>
> Farasa offers robust tools for Arabic language processing, enabling accurate tagging of grammatical categories and identification of named entities within the text.

In [171]:
pos_tagger = FarasaPOSTagger()
text = ' '.join(lemmatized_dict.values())
pos_tagged = pos_tagger.tag_segments(text)
print("POS Tagged Text:")
print(pos_tagged)

POS Tagged Text:
[('أشخاص', (['NOUN', ('M', 'P')],)), ('معني', (['ADJ', ('M', 'S')],)), ('مشارك', (['NOUN', ('M', 'S')],)), ('ريق', (['ADJ', ('M', 'S')],)), ('عمل', (['V'],)), ('مكلف', (['ADJ', ('M', 'S')],)), ('إجراء', (['NOUN', ('M', 'S')],)), ('إحصاء', (['NOUN', ('M', 'S')],)), ('عام', (['ADJ', ('M', 'S')],)), ('سكان', (['NOUN', ('M', 'P')],)), ('سكنى', (['NOUN', ('M', 'S')],)), ('2024', (['NUM', ('M', 'P')],)), ('وثائق', (['NOUN', ('F', 'P')],)), ('مطلوب', (['NOUN', ('M', 'S')],)), ('إدلاء', (['NOUN', ('M', 'S')],)), ('مشارك', (['NOUN', ('M', 'S')],)), ('أمكن', (['V'],)), ('شهاد', (['NOUN', ('F', 'P')],)), ('نجاح', (['NOUN', ('M', 'S')],)), ('دبلوم', (['NOUN', ('M', 'S')],)), ('يتم', (['V'],)), ('انتقاء', (['NOUN', ('M', 'P')],)), ('عمل', (['NOUN'],)), ('ة', (['NSUFF', ('F', 'S')],)), ('وضع', (['NOUN', ('M', 'S')],)), ('طلب', (['NOUN', ('M', 'S')],)), ('توجب', (['V'],)), ('علي', (['PREP'],)), ('فعل', (['NOUN', ('M', 'S')],)), ('كيف', (['PART'],)), ('أتم', (['NOUN', ('F', 'P')],)), 

In [172]:
ner = FarasaNamedEntityRecognizer()
ner_result = ner.recognize(text)
print("Named Entities:")
print(ner_result)

Named Entities:
أشخاص/O معني/O مشارك/O ريق/O عمل/O مكلف/O إجراء/O إحصاء/O عام/O سكان/O سكنى/O 2024/O وثائق/O مطلوب/O إدلاء/O مشارك/O أمكن/O شهاد/O نجاح/O دبلوم/O يتم/O انتقاء/O عملة/O وضع/O طلب/O توجب/O علي/O فعل/O كيف/O أتم/O إخبار/O وضع/O طلب/O ترشح/O انتقاء/O حدود/O أعداد/O مطلوب/O حاصل/O دبلوم/O شهادة/O مسلم/O طرف/O مؤسس/B-PERS جامع/I-PERS مؤسس/O تعليم/O عال/O تابع/O جامع/O تكوين/O أطر/O عليا/O تكوين/O مهن/O سنة/O أقل/O شهاد/O الباكالوريا/O شريط/O تقلي/O سن/O 20/O سنة/O فاتح/O إجراء/O طلب/O متدرب/O تابع/O دراس/O رسم/O سنة/O أولى/O ثاني/O حصول/O موظف/O أعوان/O إدارة/O عموم/O جماع/O تراب/O مستخدم/O كذا/O متقاعد/O من/O جبى/O توفر/O إدلاء/O تالي/O بطاقة/O تعريف/O وطن/O محصل/O نسب/O حامل/O شهادة/O سنة/O ثاني/O طلب/O صور/O فوتوغرافي/O ترخيص/O مسلم/O هيئة/O مشغل/O موظف/O قرار/O إحالة/O تقاعد/O متقاعد/O التزام/O مشهود/O صحة/O إمضاء/O سلط/O مختص/O شفع/O موافق/O رئيس/O إدارة/O انتمى/O إلى/O طلب/O مرشح/O إدراج/O مثبت/O شروط/O مشار/O علا/O وجوب/O منصة/O إلكترون/O مسح/O ضوء/O Pdf/O جبى/O يوم/O 

## SYNTHESIS 

Throughout this lab, I have gained valuable insights into various aspects of NLP and web scraping techniques. Here are the key takeaways from the proposed lab :

**SCRAPING FROM WEB SOURCES:**

> I learned how to use libraries like Scrapy and Beautiful Soup to extract information from web sources efficiently. Scraping data from the "www.candidature-recensement.ma" website provided hands-on experience in navigating HTML structure and extracting relevant content.<br>

**STORING RAW DATA:**

>Delving into MongoDB for data storage marked my initiation into the realm of NoSQL databases. By installing MongoDB and learning the basics of data insertion, I gained valuable insights into the fundamental principles of NoSQL database management. This hands-on experience provided me with a foundational understanding of how to effectively store and retrieve data in a non-relational database environment, setting the stage for further exploration and mastery of NoSQL technologies.

**NLP PIPELINE - DATA PREPROCESSING:**

>Implementing a comprehensive NLP pipeline, I performed various preprocessing tasks such as text cleaning, tokenization, stop words removal, normalization, stemming, lemmatization, parts-of-speech tagging, and NER. These techniques are crucial for transforming raw text data into a format suitable for analysis and modeling.

**COMPARISON BETWEEN STEMMING AND LEMMATIZATION:** 

>Through comparative analysis, I evaluated the effectiveness of stemming and lemmatization techniques, particularly in handling Arabic language text. I observed that lemmatization with the qalsadi library produced more accurate results compared to traditional stemming methods.


Overall, this lab provided me with a comprehensive understanding of NLP preprocessing and practical skills in web scraping and text processing.
